In [ ]:
# --- Cell 1: Import Libraries and Set Up Environment ---
"""
# Handwritten Character Recognition: Training Notebook

This notebook focuses on training various model architectures for handwritten character recognition.
It demonstrates how to set up a training pipeline, train different models, and save checkpoints.
"""

import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim

# Import utility modules
from data_utils_file import HandwritingDataPipeline, get_class_labels_from_directory
from src.models_util import get_model, get_model_info
from training_utils_file import train_model, load_checkpoint, test_model, plot_training_history, setup_training_components

# For reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

# Device configuration
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Create directories for saving models and visualizations
os.makedirs("model_checkpoints", exist_ok=True)
os.makedirs("training_plots", exist_ok=True)

In [ ]:
# --- Cell 2: Data Pipeline Configuration ---
"""
## Data Pipeline Configuration

Configure the data pipeline for loading and preprocessing the dataset. This pipeline handles:
- Loading images from disk
- Applying transformations and augmentations
- Splitting the dataset into training, validation, and test sets
- Creating DataLoaders for batch processing
"""

def setup_data_pipeline(data_root, image_size=(64, 64), batch_size=32, 
                        train_ratio=0.7, val_ratio=0.15, test_ratio=0.15,
                        seed=42, device=device, do_transform=True, 
                        normalization_type='imagenet'):
    """Set up the data pipeline with the specified configuration."""
    print(f"Setting up data pipeline with root: {data_root}")
    
    try:
        pipeline = HandwritingDataPipeline(
            data_root=data_root,
            image_size=image_size,
            batch_size=batch_size,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            seed=seed,
            device=device,
            do_transform=do_transform,
            normalization_type=normalization_type
        )
        
        train_loader, val_loader, test_loader = pipeline.get_loaders()
        sizes = pipeline.get_sizes()
        class_names = pipeline.get_class_labels()
        
        print(f"Dataset successfully loaded with {len(class_names)} classes")
        print(f"Split sizes: Train={sizes['train']}, Val={sizes['val']}, Test={sizes['test']}")
        
        return pipeline, train_loader, val_loader, test_loader, class_names
    
    except Exception as e:
        print(f"Error setting up data pipeline: {e}")
        return None, None, None, None, None

# Example usage (commented out, replace with actual data path):
# DATA_ROOT = "./datasets/handwritten-english/augmented_images1"
# pipeline, train_loader, val_loader, test_loader, class_names = setup_data_pipeline(DATA_ROOT)

In [ ]:
# --- Cell 3: Visualize Augmented Images ---
"""
## Visualize Augmented Images

Visualize the effect of data augmentation on training images. This helps understand
how the augmentation pipeline transforms the images, which can improve model generalization.
"""

def display_augmented_images(data_loader, num_images=5, num_augmentations=3):
    """
    Display original images and their augmented versions.
    
    Args:
        data_loader: DataLoader for the training set
        num_images: Number of unique images to display
        num_augmentations: Number of augmented versions to show per image
    """
    if data_loader is None or not hasattr(data_loader, 'dataset'):
        print("Data loader is not available or not properly initialized.")
        return
    
    # Get a batch of images
    images, labels = next(iter(data_loader))
    
    # Get class names
    if hasattr(data_loader.dataset, 'subset') and hasattr(data_loader.dataset.subset, 'dataset'):
        try:
            dataset = data_loader.dataset.subset.dataset
            if hasattr(dataset, 'class_to_idx'):
                idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
                class_names = [idx_to_class.get(i, f"Class {i}") for i in range(len(idx_to_class))]
            else:
                class_names = [f"Class {i}" for i in range(10)]  # Fallback
        except:
            class_names = [f"Class {i}" for i in range(10)]  # Fallback
    else:
        class_names = [f"Class {i}" for i in range(10)]  # Fallback
    
    # Limit to the requested number of images
    num_images = min(num_images, len(images))
    
    fig = plt.figure(figsize=(num_augmentations * 3, num_images * 3))
    
    for i in range(num_images):
        # Get the original image and label
        img = images[i].cpu()
        label = labels[i].item()
        class_name = class_names[label] if label < len(class_names) else f"Class {label}"
        
        # Display the original image
        ax = plt.subplot(num_images, num_augmentations + 1, i * (num_augmentations + 1) + 1)
        img_display = img.numpy().transpose((1, 2, 0))
        
        # Unnormalize the image for display
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img_display = std * img_display + mean
        img_display = np.clip(img_display, 0, 1)
        
        plt.imshow(img_display)
        ax.set_title(f'Original: {class_name}')
        ax.axis('off')
        
        # For each augmentation
        for j in range(num_augmentations):
            # Apply the transform again to get a different augmentation
            # This assumes we can access the original image and transform
            if hasattr(data_loader.dataset, 'transform'):
                transform = data_loader.dataset.transform
                
                # Try to get the original PIL image
                try:
                    if hasattr(data_loader.dataset, 'subset') and hasattr(data_loader.dataset.subset, 'dataset'):
                        # This is a complex structure like TransformedDataset -> Subset -> ImageFolder
                        subset = data_loader.dataset.subset
                        index = subset.indices[i]  # Get the real index in the original dataset
                        original_pil, _ = subset.dataset[index]
                        
                        # Apply transform
                        augmented = transform(original_pil)
                        
                        # Display the augmented image
                        ax = plt.subplot(num_images, num_augmentations + 1, i * (num_augmentations + 1) + j + 2)
                        aug_display = augmented.cpu().numpy().transpose((1, 2, 0))
                        
                        # Unnormalize
                        aug_display = std * aug_display + mean
                        aug_display = np.clip(aug_display, 0, 1)
                        
                        plt.imshow(aug_display)
                        ax.set_title(f'Aug {j+1}: {class_name}')
                        ax.axis('off')
                    else:
                        print("Cannot access original images for augmentation display.")
                        break
                except Exception as e:
                    print(f"Error displaying augmentation: {e}")
                    break
            else:
                print("Cannot access the transform for augmentation display.")
                break
    
    plt.tight_layout()
    plt.savefig("training_plots/augmented_images.png")
    plt.show()

# Example usage (depends on the data pipeline being initialized):
# display_augmented_images(train_loader, num_images=4, num_augmentations=3)

In [ ]:
# --- Cell 4: Model Training Configuration ---
"""
## Model Training Configuration

Configure and train different model architectures. This section demonstrates:
- Setting up different model architectures
- Configuring optimizers and learning rate schedulers
- Training models with the configured pipeline
- Saving model checkpoints
"""

def train_and_evaluate_model(model_name, train_loader, val_loader, test_loader, 
                           num_classes, num_epochs=25, learning_rate=0.001,
                           optimizer_name='adam', scheduler_name='cosine',
                           save_dir='model_checkpoints', pretrained=True):
    """
    Train and evaluate a model with the specified configuration.
    
    Args:
        model_name: Name of the model architecture to use
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        test_loader: DataLoader for test data
        num_classes: Number of output classes
        num_epochs: Number of training epochs
        learning_rate: Initial learning rate
        optimizer_name: Name of optimizer to use
        scheduler_name: Name of learning rate scheduler to use
        save_dir: Directory to save model checkpoints
        pretrained: Whether to use pretrained weights (for transfer learning models)
    
    Returns:
        tuple: (trained_model, history)
    """
    print(f"\n{'='*50}")
    print(f"Training {model_name} model")
    print(f"{'='*50}")
    
    # Get model info
    model_info = get_model_info(model_name)
    print(f"Model: {model_info['name']}")
    print(f"Description: {model_info['description']}")
    
    # Create model-specific directory
    model_dir = os.path.join(save_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    try:
        # Initialize model
        model = get_model(model_name, num_classes, device, pretrained)
        print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
        
        # Setup optimizer and scheduler
        optimizer_config = {
            'name': optimizer_name,
            'lr': learning_rate,
            'weight_decay': 1e-4 if model_name.startswith('vgg') else 0
        }
        
        scheduler_config = None
        if scheduler_name:
            if scheduler_name == 'cosine':
                scheduler_config = {
                    'name': 'cosine',
                    'T_max': num_epochs,
                    'eta_min': 1e-6
                }
            elif scheduler_name == 'plateau':
                scheduler_config = {
                    'name': 'plateau',
                    'mode': 'min',
                    'factor': 0.1,
                    'patience': 3,
                    'verbose': True
                }
            elif scheduler_name == 'onecycle':
                scheduler_config = {
                    'name': 'onecycle',
                    'max_lr': learning_rate * 10,
                    'epochs': num_epochs,
                    'steps_per_epoch': len(train_loader)
                }
        
        # Setup training components
        criterion, optimizer, scheduler = setup_training_components(
            model, optimizer_config, scheduler_config, device
        )
        
        # Train the model
        print(f"Starting training for {num_epochs} epochs...")
        start_time = time.time()
        
        trained_model, history = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            device=device,
            save_dir=model_dir,
            model_name=model_name,
            verbose=True,
            save_every_n_epochs=5
        )
        
        training_time = time.time() - start_time
        print(f"Training completed in {training_time/60:.2f} minutes")
        
        # Plot training history
        print("Plotting training history...")
        plot_training_history(
            history, 
            save_path=os.path.join(model_dir, f"{model_name}_training_history.png")
        )
        
        # Evaluate on test set
        print("\nEvaluating on test set...")
        test_loss, test_acc = test_model(
            model=trained_model,
            test_loader=test_loader,
            criterion=criterion,
            device=device,
            verbose=True
        )
        
        print(f"Test Accuracy: {test_acc:.4f}")
        
        # Save final results
        results = {
            'model_name': model_name,
            'num_epochs': num_epochs,
            'training_time': training_time,
            'final_train_loss': history['train_loss'][-1],
            'final_train_acc': history['train_acc'][-1],
            'final_val_loss': history['val_loss'][-1] if history['val_loss'] else None,
            'final_val_acc': history['val_acc'][-1] if history['val_acc'] else None,
            'test_loss': test_loss,
            'test_acc': test_acc
        }
        
        # Save results to file
        import json
        with open(os.path.join(model_dir, f"{model_name}_results.json"), 'w') as f:
            json.dump(results, f, indent=4)
        
        return trained_model, history
    
    except Exception as e:
        print(f"Error during model training: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Example usage (depends on the data pipeline being initialized):
# model, history = train_and_evaluate_model(
#     model_name='improved_cnn',
#     train_loader=train_loader,
#     val_loader=val_loader,
#     test_loader=test_loader,
#     num_classes=len(class_names),
#     num_epochs=20,
#     learning_rate=0.001,
#     optimizer_name='adam',
#     scheduler_name='cosine'
# )

In [ ]:
# --- Cell 5: Train Multiple Models ---
"""
## Train Multiple Models

This section demonstrates how to train multiple model architectures with different
configurations to compare their performance.
"""

def train_multiple_models(train_loader, val_loader, test_loader, class_names, 
                         configs=None, save_dir='model_checkpoints'):
    """
    Train multiple models with different configurations.
    
    Args:
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        test_loader: DataLoader for test data
        class_names: List of class names
        configs: List of model configurations
        save_dir: Directory to save model checkpoints
    
    Returns:
        dict: Results for all models
    """
    if configs is None:
        configs = [
            {
                'model_name': 'basic_cnn',
                'num_epochs': 15,
                'learning_rate': 0.001,
                'optimizer_name': 'adam',
                'scheduler_name': 'cosine',
                'pretrained': False
            },
            {
                'model_name': 'improved_cnn',
                'num_epochs': 20,
                'learning_rate': 0.001,
                'optimizer_name': 'adam',
                'scheduler_name': 'cosine',
                'pretrained': False
            },
            {
                'model_name': 'vgg19',
                'num_epochs': 10,
                'learning_rate': 0.0001,
                'optimizer_name': 'adam',
                'scheduler_name': 'plateau',
                'pretrained': True
            }
        ]
    
    num_classes = len(class_names)
    results = {}
    
    for config in configs:
        model_name = config['model_name']
        print(f"\n\nTraining model: {model_name}")
        
        model, history = train_and_evaluate_model(
            model_name=model_name,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            num_classes=num_classes,
            num_epochs=config.get('num_epochs', 20),
            learning_rate=config.get('learning_rate', 0.001),
            optimizer_name=config.get('optimizer_name', 'adam'),
            scheduler_name=config.get('scheduler_name', 'cosine'),
            save_dir=save_dir,
            pretrained=config.get('pretrained', False)
        )
        
        results[model_name] = {
            'model': model,
            'history': history,
            'config': config
        }
    
    # Compare results
    print("\n\n" + "="*50)
    print("Model Comparison Summary")
    print("="*50)
    
    for model_name, result in results.items():
        if result['history'] is not None:
            config = result['config']
            history = result['history']
            
            best_val_acc = max(history['val_acc']) if history['val_acc'] else 0
            final_train_acc = history['train_acc'][-1] if history['train_acc'] else 0
            
            print(f"Model: {model_name}")
            print(f"  Config: {config}")
            print(f"  Best Validation Accuracy: {best_val_acc:.4f}")
            print(f"  Final Training Accuracy: {final_train_acc:.4f}")
            print("-"*50)
    
    return results

# Example usage (depends on the data pipeline being initialized):
# results = train_multiple_models(train_loader, val_loader, test_loader, class_names)

In [ ]:
# --- Cell 6: Fine-tuning and Advanced Training ---
"""
## Fine-tuning and Advanced Training

This section demonstrates more advanced training techniques:
- Fine-tuning pretrained models
- Freezing and unfreezing layers
- Learning rate scheduling strategies
"""

def freeze_layers(model, num_layers_to_freeze):
    """
    Freeze the first num_layers_to_freeze layers of the model.
    
    Args:
        model: PyTorch model
        num_layers_to_freeze: Number of layers to freeze
    """
    if hasattr(model, 'features') and isinstance(model.features, nn.Sequential):
        # Count actual layers (Conv2d, Linear, etc.) if features is a sequential block
        layer_idx = 0
        for child in model.features.children():
            if isinstance(child, (nn.Conv2d, nn.Linear, nn.BatchNorm2d)):
                if layer_idx < num_layers_to_freeze:
                    for param in child.parameters():
                        param.requires_grad = False
                layer_idx += 1
        print(f"Froze {min(num_layers_to_freeze, layer_idx)} layers in model.features.")
    else:
        # Fallback for generic parameters
        params = list(model.parameters())
        actual_layers_to_freeze = min(num_layers_to_freeze, len(params))
        for i, param in enumerate(params):
            if i < actual_layers_to_freeze:
                param.requires_grad = False
        print(f"Froze first {actual_layers_to_freeze} parameter groups of the model.")

def fine_tune_model(model_name, train_loader, val_loader, test_loader, class_names,
                   initial_epochs=5, fine_tune_epochs=15, save_dir='model_checkpoints',
                   freeze_layers_count=None, unfreeze_after_epoch=None):
    """
    Fine-tune a pretrained model with a two-phase training approach.
    
    Args:
        model_name: Name of the model architecture to use
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        test_loader: DataLoader for test data
        class_names: List of class names
        initial_epochs: Number of epochs for initial training phase
        fine_tune_epochs: Number of epochs for fine-tuning phase
        save_dir: Directory to save model checkpoints
        freeze_layers_count: Number of layers to freeze initially
        unfreeze_after_epoch: Epoch after which to unfreeze all layers
    
    Returns:
        tuple: (fine_tuned_model, history)
    """
    print(f"\n{'='*50}")
    print(f"Fine-tuning {model_name} model")
    print(f"{'='*50}")
    
    model_dir = os.path.join(save_dir, f"{model_name}_finetuned")
    os.makedirs(model_dir, exist_ok=True)
    
    num_classes = len(class_names)
    
    try:
        # Initialize model with pretrained weights
        model = get_model(model_name, num_classes, device, pretrained=True)
        print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
        
        # Phase 1: Train with frozen layers (if specified)
        if freeze_layers_count is not None and freeze_layers_count > 0:
            print(f"\nPhase 1: Training with {freeze_layers_count} frozen layers for {initial_epochs} epochs")
            freeze_layers(model, freeze_layers_count)
            
            # Setup optimizer and scheduler for phase 1
            optimizer_config = {
                'name': 'adam',
                'lr': 0.0001,  # Lower learning rate for pretrained features
                'weight_decay': 1e-4
            }
            
            scheduler_config = {
                'name': 'cosine',
                'T_max': initial_epochs,
                'eta_min': 1e-6
            }
            
            criterion, optimizer, scheduler = setup_training_components(
                model, optimizer_config, scheduler_config, device
            )
            
            # Train with frozen layers
            model, phase1_history = train_model(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                criterion=criterion,
                optimizer=optimizer,
                scheduler=scheduler,
                num_epochs=initial_epochs,
                device=device,
                save_dir=model_dir,
                model_name=f"{model_name}_phase1",
                verbose=True
            )
            
            print(f"Phase 1 completed. Best validation accuracy: {max(phase1_history['val_acc']):.4f}")
        
        # Phase 2: Fine-tune all layers
        print(f"\nPhase 2: Fine-tuning all layers for {fine_tune_epochs} epochs")
        
        # Unfreeze all layers
        for param in model.parameters():
            param.requires_grad = True
        
        # Setup optimizer and scheduler for phase 2
        optimizer_config = {
            'name': 'adam',
            'lr': 0.00005,  # Even lower learning rate for fine-tuning
            'weight_decay': 1e-4
        }
        
        scheduler_config = {
            'name': 'cosine',
            'T_max': fine_tune_epochs,
            'eta_min': 1e-7
        }
        
        criterion, optimizer, scheduler = setup_training_components(
            model, optimizer_config, scheduler_config, device
        )
        
        # Fine-tune the model
        fine_tuned_model, phase2_history = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=fine_tune_epochs,
            device=device,
            save_dir=model_dir,
            model_name=f"{model_name}_finetuned",
            verbose=True
        )
        
        # Combine histories from both phases
        if freeze_layers_count is not None and freeze_layers_count > 0:
            combined_history = {
                'train_loss': phase1_history['train_loss'] + phase2_history['train_loss'],
                'train_acc': phase1_history['train_acc'] + phase2_history['train_acc'],
                'val_loss': phase1_history['val_loss'] + phase2_history['val_loss'],
                'val_acc': phase1_history['val_acc'] + phase2_history['val_acc'],
                'learning_rates': phase1_history['learning_rates'] + phase2_history['learning_rates']
            }
        else:
            combined_history = phase2_history
        
        # Plot combined training history
        plot_training_history(
            combined_history, 
            save_path=os.path.join(model_dir, f"{model_name}_finetuned_history.png")
        )
        
        # Evaluate on test set
        print("\nEvaluating fine-tuned model on test set...")
        test_loss, test_acc = test_model(
            model=fine_tuned_model,
            test_loader=test_loader,
            criterion=criterion,
            device=device,
            verbose=True
        )
        
        print(f"Test Accuracy: {test_acc:.4f}")
        
        # Save final results
        results = {
            'model_name': f"{model_name}_finetuned",
            'initial_epochs': initial_epochs if freeze_layers_count is not None else 0,
            'fine_tune_epochs': fine_tune_epochs,
            'frozen_layers': freeze_layers_count,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'best_val_acc': max(combined_history['val_acc']) if combined_history['val_acc'] else 0
        }
        
        # Save results to file
        import json
        with open(os.path.join(model_dir, f"{model_name}_finetuned_results.json"), 'w') as f:
            json.dump(results, f, indent=4)
        
        return fine_tuned_model, combined_history
    
    except Exception as e:
        print(f"Error during model fine-tuning: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Example usage (depends on the data pipeline being initialized):
# fine_tuned_model, history = fine_tune_model(
#     model_name='vgg19',
#     train_loader=train_loader,
#     val_loader=val_loader,
#     test_loader=test_loader,
#     class_names=class_names,
#     initial_epochs=5,
#     fine_tune_epochs=15,
#     freeze_layers_count=20
# )

In [ ]:
# --- Cell 7: Complete Training Pipeline Example ---
"""
## Complete Training Pipeline Example

This section demonstrates a complete training pipeline from data loading to model evaluation.
Follow this example to train models on your own dataset.
"""

def run_complete_training_pipeline(data_root, image_size=(64, 64), batch_size=32,
                                  model_name='improved_cnn', num_epochs=20,
                                  learning_rate=0.001, optimizer_name='adam',
                                  scheduler_name='cosine', pretrained=False,
                                  save_dir='model_checkpoints'):
    """
    Run a complete training pipeline from data loading to model evaluation.
    
    Args:
        data_root: Path to the dataset root directory
        image_size: Input image size
        batch_size: Batch size for training
        model_name: Model architecture to use
        num_epochs: Number of training epochs
        learning_rate: Initial learning rate
        optimizer_name: Optimizer to use
        scheduler_name: Learning rate scheduler to use
        pretrained: Whether to use pretrained weights
        save_dir: Directory to save model checkpoints
        
    Returns:
        tuple: (trained_model, history, class_names)
    """
    print(f"Running complete training pipeline for {model_name} model")
    
    # Step 1: Set up data pipeline
    print("\nStep 1: Setting up data pipeline...")
    pipeline, train_loader, val_loader, test_loader, class_names = setup_data_pipeline(
        data_root=data_root,
        image_size=image_size,
        batch_size=batch_size,
        do_transform=True
    )
    
    if pipeline is None:
        print("Failed to set up data pipeline. Aborting.")
        return None, None, None
    
    # Step 2: Visualize augmented images
    print("\nStep 2: Visualizing augmented images...")
    display_augmented_images(train_loader, num_images=3, num_augmentations=3)
    
    # Step 3: Train the model
    print("\nStep 3: Training the model...")
    trained_model, history = train_and_evaluate_model(
        model_name=model_name,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        num_classes=len(class_names),
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        scheduler_name=scheduler_name,
        save_dir=save_dir,
        pretrained=pretrained
    )
    
    if trained_model is None:
        print("Failed to train the model. Aborting.")
        return None, None, class_names
    
    # Step 4: Save model architecture info
    print("\nStep 4: Saving model architecture info...")
    model_dir = os.path.join(save_dir, model_name)
    
    # Save class names
    with open(os.path.join(model_dir, 'class_names.txt'), 'w') as f:
        for name in class_names:
            f.write(f"{name}\n")
    
    # Save model architecture info
    model_info = get_model_info(model_name)
    with open(os.path.join(model_dir, 'model_info.txt'), 'w') as f:
        f.write(f"Model: {model_info['name']}\n")
        f.write(f"Description: {model_info['description']}\n")
        f.write(f"Parameters: {model_info['parameters']}\n")
        f.write(f"Training Time: {model_info['training_time']}\n")
        f.write(f"Expected Accuracy: {model_info['accuracy']}\n")
    
    print(f"Training pipeline completed successfully. Model saved to {model_dir}")
    return trained_model, history, class_names

# Usage example (uncomment and modify with your actual data path):
"""
DATA_ROOT = "./your_dataset_path"
trained_model, history, class_names = run_complete_training_pipeline(
    data_root=DATA_ROOT,
    model_name='improved_cnn',
    num_epochs=20
)
"""

In [ ]:
# --- Cell 8: Run Training (User Code) ---
"""
## Run Training

This is where you run the actual training pipeline with your dataset.
Uncomment and modify the code below to train models on your own dataset.
"""

# Define your dataset path
# DATA_ROOT = "./datasets/handwritten-english/augmented_images1"

# Option 1: Run the complete pipeline for a single model
"""
trained_model, history, class_names = run_complete_training_pipeline(
    data_root=DATA_ROOT,
    model_name='improved_cnn',
    num_epochs=20,
    batch_size=32,
    learning_rate=0.001,
    optimizer_name='adam',
    scheduler_name='cosine'
)
"""

# Option 2: Train multiple models
"""
# First set up the data pipeline
pipeline, train_loader, val_loader, test_loader, class_names = setup_data_pipeline(
    data_root=DATA_ROOT,
    batch_size=32,
    do_transform=True
)

# Define configurations for multiple models
configs = [
    {
        'model_name': 'basic_cnn',
        'num_epochs': 15,
        'learning_rate': 0.001,
        'optimizer_name': 'adam',
        'scheduler_name': 'cosine',
        'pretrained': False
    },
    {
        'model_name': 'improved_cnn',
        'num_epochs': 20,
        'learning_rate': 0.001,
        'optimizer_name': 'adam',
        'scheduler_name': 'cosine',
        'pretrained': False
    },
    {
        'model_name': 'vgg19',
        'num_epochs': 10,
        'learning_rate': 0.0001,
        'optimizer_name': 'adam',
        'scheduler_name': 'plateau',
        'pretrained': True
    }
]

# Train all models
results = train_multiple_models(
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=class_names,
    configs=configs
)
"""

# Option 3: Fine-tune a pretrained model
"""
# First set up the data pipeline
pipeline, train_loader, val_loader, test_loader, class_names = setup_data_pipeline(
    data_root=DATA_ROOT,
    batch_size=32,
    do_transform=True
)

# Fine-tune a VGG19 model
fine_tuned_model, history = fine_tune_model(
    model_name='vgg19',
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=class_names,
    initial_epochs=5,
    fine_tune_epochs=15,
    freeze_layers_count=20
)
"""

print("This notebook is ready for training handwritten character recognition models.")
print("Uncomment one of the training options above and run this cell to start training.")